# 1. Imports

In [1]:
from sklearn.decomposition import PCA
import matplotlib as mpl 
import numpy as np
import os
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler


from codes.utils import stratified_train_test_group_kfold
from codes.utils import model_test_classification

from codes.classification_codes import gbc_gridsearch



0.10.1


# 2. Data

In [2]:
with open("Data/MoS2_Analysis_Data_trained2", "rb") as fp:   # Unpickling
    MoS2_Proj1_Class_Data = pickle.load(fp)

#features = MoS2_ImageNet_100_data[0]
df = pd.DataFrame(MoS2_Proj1_Class_Data)
print(df.keys())
T_dict= {900:0, 950:1, 1000:2}
T_target = np.array([T_dict[item] for item in df['T']])

features = np.array(list(df['ImageNet']))
sampleId = np.array(list(df['sampleId']))

X = features
Y = np.array(T_target)
groups = np.array(sampleId)


train_val_groups, train_val_X, train_val_Y, test_X, test_Y = stratified_train_test_group_kfold(X, Y, groups, n_splits=10, test_fold=0)

train_val_Y = train_val_Y.flatten()
test_Y = test_Y.flatten()

print(train_val_X.shape)
print(train_val_Y.shape)
print(test_X.shape)
print(test_Y.shape)


oversample = RandomOverSampler(sampling_strategy='not majority')

scaler = StandardScaler().fit(train_val_X)
train_val_X = scaler.transform(train_val_X)
test_X = scaler.transform(test_X)

Index(['sampleId', 'sampleLabel', 'image', 'T', 'ImageNet', 'MicroNet'], dtype='object')


In [3]:
! mkdir -p 'Model/Class/TrainedImageNet/Up/'

# 3. Runing Cross-Validation

In [5]:
N_estimators = [i for i in range(100, 700, 100)]

Learning_rate = [0.0, 0.001, 0.01, 0.1, 1]

Min_samples_split = [i for i in range(2, 15, 3)]

Max_depth = [6, 7, 8]#8, 9, 10]



def cross_10_folds_gbc(train_val_X, train_val_Y):
    
    best_train = []
    best_val = []
    best_variables = []
    for fold in range(10):
        group, train_X, train_Y, val_X, val_Y = stratified_train_test_group_kfold(train_val_X, train_val_Y, train_val_groups, n_splits=10, test_fold=fold)
        
        model_path = f'classificatioin/ImageNet/aug3/GBC_{fold}_model.sav'
        
        train_X, train_Y = oversample.fit_resample(train_X, train_Y)
        variables_best, performance_best = gbc_gridsearch(train_X, train_Y, val_X, val_Y, N_estimators, Learning_rate, Min_samples_split,Max_depth,fold, model_path)
        best_train.append(performance_best['accuracy_train'])
        best_val.append(performance_best['accuracy_val'])
        best_variables.append(variables_best)
        print(f'fold: {fold} done!')
    return best_train, best_val, best_variables


best_train, best_val, best_variables = cross_10_folds_gbc(train_val_X, train_val_Y)
print(best_train)
print(best_val)
print(f'train_acc_mean: {np.mean(best_train) :.3f}, std: {np.std(best_train) :.3f}, val_acc_mean: {np.mean(best_val):.3f}, std: {np.std(best_val):.3f}')

<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.125
best_train_acc: 0.9904761904761905, best_val_acc: 0.625
best_train_acc: 0.9968253968253968, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.75
best_train_acc: 1.0, best_val_acc: 0.7916666666666666
fold: 0 done!
<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.125
best_train_acc: 0.9968553459119497, best_val_acc: 0.5416666666666666
best_train_acc: 1.0, best_val_acc: 0.5833333333333334
best_train_acc: 1.0, best_val_acc: 0.625
best_train_acc: 1.0, best_val_acc: 0.6666666666666666
fold: 1 done!
<class 'generator'>
best_train_acc: 0.3333333333333333, best_val_acc: 0.125
best_train_acc: 0.9968553459119497, best_val_acc: 0.625
best_train_acc: 1.0, best_val_acc: 0.6666666666666666
best_train_acc: 1.0, best_val_acc: 0.7083333333333334
best_train_acc: 1.0, best_val_acc: 0.75
best_train_acc: 1.0, best_val_acc: 0.7916666666666666
best_train_acc: 1.0, best_val_acc: 0.833333333333

# 4. Model Testing

In [6]:
best_fold = 3

model_path = f'classificatioin/ImageNet/aug3/GBC_{best_fold}_model.sav'

# Fitting different train in the same hyperparameters
best_test, confusion_matrix_test = model_test_classification(train_val_X, train_val_Y, train_val_groups,test_X, test_Y, best_fold, model_path)



print(best_test)
print(f'test_acc_mean: {np.mean(best_test) :.3f}, std: {np.std(best_test) :.3f}')

print('5th confusion matrix: ', [list(item) for item in confusion_matrix_test[4]])

<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
<class 'generator'>
[0.6666666666666666, 0.6666666666666666, 0.6296296296296297, 0.7037037037037037, 0.6296296296296297, 0.6296296296296297, 0.6296296296296297, 0.6296296296296297, 0.7407407407407407, 0.6296296296296297]
test_acc_mean: 0.656, std: 0.037
5th confusion matrix:  [[1, 2, 0], [0, 8, 5], [1, 2, 8]]


In [7]:
cm_list = []
for cm in confusion_matrix_test:
    cm_list.append([list(item) for item in cm])
#print(cm_list)

print(len(cm_list))
print([list(item) for item in np.mean(cm_list, axis=0)])

10
[[0.3, 2.3, 0.4], [0.6, 9.1, 3.3], [0.5, 2.2, 8.3]]


In [3]:
best_fold = 3

#PATH = os.path.join('Model', model_path)
PATH = f'Model/Class/TrainedImageNet/Up/GBC_{best_fold}_model.sav'
loaded_model = pickle.load(open(PATH, 'rb'))
loaded_model.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.01,
 'loss': 'log_loss',
 'max_depth': 8,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 11,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_iter_no_change': None,
 'random_state': 1,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}